- **Author:** Kandimalla Hemanth
- **E-mail:** speechcodehemanth@gmail.com

# Efficient Deep Learning with Accelerate: A Quick Overview

### `DataLoaderConfiguration`
| Argument | Description | Default |
|---|---|---|
| **`split_batches`** | Whether to split batches across devices. | `False` |
| **`dispatch_batches`** | Whether to iterate dataloader on main process only. | `True` for `IterableDataset`, `False` otherwise. |
| **`even_batches`** | Whether to duplicate samples to ensure equal batch division. | `True` |
| **`use_seedable_sampler`** | Whether to use a fully seedable random sampler. | `False` |
| **`data_seed`** | Seed to use for the generator when `use_seedable_sampler` is True. | `None` |
| **`non_blocking`** | Whether to use non-blocking host-to-device transfers. | `False` |
| **`use_stateful_dataloader`** | Whether to use a `torchdata.StatefulDataLoader`. | `False` |

Detailed explanations for each argument are provided below:

*   **`split_batches`** : This argument determines how batches are distributed across devices. If set to  `True`, batches yielded by the dataloaders are split across devices, resulting in the same batch size being used on all processes. However, the batch size must be a multiple of the number of processes being used. If set to  `False`, the actual batch size used is the one specified in the script multiplied by the number of processes.

*   **`dispatch_batches`** : When set to  `True`, the dataloader is iterated only on the main process, and the resulting batches are split and broadcast to each process. This is the default behavior for `DataLoader` instances based on `IterableDataset`. For other types of datasets, the default is  `False`.

*   **`even_batches`** : If the total batch size across all processes doesn't divide the dataset evenly, setting this argument to  `True`  will duplicate samples at the start of the dataset to ensure that the batch can be divided equally among all workers. This ensures consistent batch sizes across all processes, which can be beneficial for certain training scenarios. The default value is  `True`.

*   **`use_seedable_sampler`** : Enabling this option ensures that training results are fully reproducible by employing a fully seedable random sampler (`data_loader.SeedableRandomSampler`). This technique ensures that the data is shuffled in the same order across different runs, leading to consistent results. While seed-to-seed results may vary slightly, the overall differences are generally negligible when comparing results across multiple seeds. For optimal reproducibility, it's recommended to use `use_seedable_sampler` in conjunction with `~utils.set_seed`.

*   **`data_seed`** : When `use_seedable_sampler` is set to `True`, this argument specifies the seed to be used for the underlying generator. If set to `None`, the generator will use the current default seed from PyTorch. This provides control over the random shuffling of the dataset when using a seedable sampler.

*   **`non_blocking`** : By setting this argument to `True`, the prepared dataloader will utilize non-blocking host-to-device transfers. This allows for improved overlap between dataloader communication and computation, potentially reducing training time. For this to work effectively, it's recommended to set `pin_memory` to `True` in the dataloader.

*   **`use_stateful_dataloader`** : This argument, if set to  `True`, configures the dataloader to be backed by a `torchdata.StatefulDataLoader`. This feature requires the installation of `torchdata` version 0.8.0 or higher. `StatefulDataLoader` offers advantages in certain scenarios, such as handling stateful datasets or improving performance.


###  `Accelerator`
| Argument | Description | Default |
|---|---|---|
| **`device_placement`** | Determines whether the accelerator should automatically place objects on the correct device. | `True` |
| **`mixed_precision`** | Specifies the mixed precision mode to use. | Value of the `ACCELERATE_MIXED_PRECISION` environment variable, or the default in the accelerate config. |
| **`gradient_accumulation_steps`** | Number of steps to accumulate gradients before updating model weights. | `1` |
| **`cpu`** | Forces the script to run on the CPU, ignoring available GPUs. | `False` |
| **`dataloader_config`** | Configuration for handling dataloaders in distributed settings. See the previous response for more details on this argument. | `DataLoaderConfiguration()` |
| **`deepspeed_plugin`** |  Allows customization of DeepSpeed-related arguments. Can be a `DeepSpeedPlugin` object or a dictionary of `DeepSpeedPlugin` objects. | `DeepSpeedPlugin()` if the environment variable `ACCELERATE_USE_DEEPSPEED` is "true", `None` otherwise. |
| **`fsdp_plugin`** | Allows customization of Fully Sharded Data Parallel (FSDP) arguments. This should be a `FullyShardedDataParallelPlugin` object. | `None` |
| **`megatron_lm_plugin`** | Allows customization of Megatron-LM arguments. This should be a `MegatronLMPlugin` object. | `None` |
| **`rng_types`** |  Specifies the random number generators to synchronize at each iteration. | `["generator"]` |
| **`log_with`** |  List of loggers to set up for experiment tracking. | `None` |
| **`project_dir`** | Path to a directory to store logs and checkpoints. | `None` |
| **`project_config`** | Configuration for managing the saving of the training state. | `None` |
| **`gradient_accumulation_plugin`** | Allows fine-grained control over gradient accumulation. | `None` |
| **`step_scheduler_with_optimizer`** |  Determines if the learning rate scheduler is updated at the same time as the optimizer. | `True` |
| **`kwargs_handlers`** | Allows passing in keyword arguments specific to the backend. This should be a list of `KwargsHandler` objects. | `None` |
| **`dynamo_backend`** |  Specifies the backend to use for TorchDynamo. | `None` |
| **`dynamo_plugin`** | Allows customization of TorchDynamo related arguments. | `TorchDynamoPlugin()` |

These arguments offer significant control over the behavior of the accelerator, allowing you to configure it for different hardware, distributed training strategies, mixed-precision training, and other settings. For example:

*   **`device_placement`:** This argument controls whether the accelerator will place the model, optimizer, and dataloader on the appropriate device. If you have multiple GPUs, this argument will place these objects on the correct GPU.
*   **`mixed_precision`**: This argument lets you enable mixed precision training. This can speed up training, but may require careful tuning of the learning rate and other hyperparameters.
*   **`gradient_accumulation_steps`**: This argument lets you specify how many batches to process before updating the model weights. This can be useful when training on very large batches, or when using a limited amount of GPU memory.

For more details on any specific argument, please refer to the documentation for the [`Accelerator`](https://huggingface.co/docs/accelerate/en/package_reference/accelerator) class in the Hugging Face Accelerate library.


```python
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from accelerate import Accelerator,  DataLoaderConfiguration

# Define the model
class SimpleModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(28*28, 10)
    def forward(self, x):
        return self.linear(x.view(x.size(0), -1))
model = SimpleModel()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Define the learning rate scheduler
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 0.95 ** epoch)

# Create the dataloaders
train_dataset = MNIST(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root='./data', train=False, download=True, transform=ToTensor())
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


# Initialize the Accelerator
accelerator = Accelerator(
    gradient_accumulation_steps=2, # accumulate gradients for 2 steps
    mixed_precision="fp16", # Enable fp16 mixed precision
    dataloader_config=DataLoaderConfiguration(split_batches=True), # Split batches across devices
    
)

# Prepare the model, optimizer, and dataloaders
model, optimizer, train_dataloader, test_dataloader, scheduler  = accelerator.prepare(
    model, optimizer, train_dataloader, test_dataloader, scheduler
)

# Training loop
for epoch in range(5):
    for batch_idx, (inputs, targets) in enumerate(train_dataloader):
        # Forward pass
        outputs = model(inputs)
        loss = torch.nn.functional.cross_entropy(outputs, targets)
        
        # Backward pass
        accelerator.backward(loss)

        # Update weights
        if accelerator.sync_gradients:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

# Evaluate the model
with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(test_dataloader):
        outputs = model(inputs)
        # Gather results across processes
        all_outputs = accelerator.gather(outputs)
        all_targets = accelerator.gather(targets)

```

**Explanation**

*   **Import Necessary Libraries**: The code starts by importing essential modules from  `torch`,  `torchvision`, and  `accelerate`.
*   **Model, Optimizer, Scheduler, Dataloaders**: It defines a basic neural network model (`SimpleModel`), an optimizer (`torch.optim.Adam`), a learning rate scheduler (`torch.optim.lr_scheduler.LambdaLR`), and creates dataloaders for training and testing using the MNIST dataset.
*   **Accelerator Initialization**: An `Accelerator` instance is created, enabling features like gradient accumulation (`gradient_accumulation_steps`), mixed precision training (`mixed_precision`), and batch splitting (`dataloader_config`).
*   **Preparation with  `accelerator.prepare()`**: The `prepare`  method readies the model, optimizer, dataloaders, and scheduler for distributed training and mixed precision.
*   **Training Loop**: A standard training loop iterates through epochs and batches, performing the forward pass, calculating the loss, and using  `accelerator.backward()`  for backpropagation. The optimizer steps and scheduler adjustments are done when gradients are synchronized (`accelerator.sync_gradients`).
*   **Evaluation**: The model is evaluated on the test set, and `accelerator.gather()`  is used to collect predictions and targets from different processes.

**Key Features of  `Accelerator`  used:**

*   **`prepare()`**: This function is central to the `Accelerator` functionality, as it prepares the model, optimizer, dataloaders, and learning rate scheduler for distributed training and mixed precision. It handles device placement, wraps objects for distributed operations, and enables the specified mixed precision mode.
*   **`backward()`**: This method replaces the standard  `loss.backward()`  call and ensures proper gradient scaling in distributed and mixed precision settings. It's crucial for correct backpropagation, especially when using techniques like gradient accumulation.
*   **`sync_gradients`**: This attribute indicates whether gradients are currently synchronized across processes. It's used to control when the optimizer updates the model weights, ensuring proper gradient accumulation and synchronization.
*   **`gather()`**: This function collects tensors from all processes and concatenates them along the first dimension, enabling aggregation of results, particularly in evaluation or when gathering all predictions or labels.

The  `accelerator.prepare()`  method sets up the model and other objects for distributed training, and the other features handle tasks like gradient calculation, synchronization, and gathering of distributed data.


```python
import argparse
import os
import torch
import evaluate
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    set_seed,
)
from datasets import load_dataset
from accelerate import Accelerator

def training(accelerator, model, tokenizer, train_dataloader, optimizer, lr_scheduler, config, args):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
    return total_loss / len(train_dataloader)

def evaluation(accelerator, model, eval_dataloader, metric):
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        metric.add_batch(
            predictions=accelerator.gather_for_metrics(predictions),
            references=accelerator.gather_for_metrics(batch["labels"]),
        )
    return metric.compute()

def testing(model, test_dataloader, accelerator):
    model.eval()
    predictions = []
    for batch in test_dataloader:
        with torch.no_grad():
            outputs = model(**batch)
        preds = accelerator.gather_for_metrics(outputs.logits.argmax(dim=-1))
        predictions.extend(preds.tolist())
    return predictions

def inference(model, tokenizer, prompt, accelerator, max_new_tokens=50, temperature=0.7, top_k=50, top_p=0.95):
    model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(accelerator.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_k=top_k, top_p=top_p)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def main():
    parser = argparse.ArgumentParser(description="Training, evaluation, testing, and inference script.")
    parser.add_argument("--mode", type=str, required=True, choices=["train", "evaluate", "test", "infer"])
    parser.add_argument("--model_name", type=str, default="bert-base-cased")
    parser.add_argument("--dataset_name", type=str, default="glue")
    parser.add_argument("--task_name", type=str, default="mrpc")
    parser.add_argument("--num_epochs", type=int, default=3)
    parser.add_argument("--batch_size", type=int, default=16)
    parser.add_argument("--learning_rate", type=float, default=2e-5)
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--mixed_precision", type=str, default="no", choices=["no", "fp16", "bf16"])
    parser.add_argument("--output_dir", type=str, default="output")
    parser.add_argument("--checkpoint", type=str, help="Path to a checkpoint for evaluation/testing/inference")
    parser.add_argument("--prompt", type=str, help="Prompt for inference mode")
    args = parser.parse_args()

    config = {
        "num_epochs": args.num_epochs,
        "batch_size": args.batch_size,
        "learning_rate": args.learning_rate,
    }

    accelerator = Accelerator(mixed_precision=args.mixed_precision)
    set_seed(args.seed)

    if args.mode == "infer":
        if not args.checkpoint or not args.prompt:
            raise ValueError("Checkpoint path and prompt are required for inference.")
        model = AutoModelForCausalLM.from_pretrained(args.checkpoint)
        tokenizer = AutoTokenizer.from_pretrained(args.checkpoint)
        model, tokenizer = accelerator.prepare(model, tokenizer)
        generated_text = inference(model, tokenizer, args.prompt, accelerator)
        print(f"Generated Text:\n{generated_text}")
        return
    
    tokenizer = AutoTokenizer.from_pretrained(args.model_name)
    datasets = load_dataset(args.dataset_name, args.task_name)

    def tokenize_function(examples):
        return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding="max_length", max_length=128)
    
    with accelerator.main_process_first():
      tokenized_datasets = datasets.map(tokenize_function, batched=True)
      tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

    train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=config["batch_size"])
    eval_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=config["batch_size"])

    if args.mode == "train" or args.mode == "evaluate":
        metric = evaluate.load(args.dataset_name, args.task_name)
        model = AutoModelForSequenceClassification.from_pretrained(args.model_name, num_labels=2) # Assuming binary classification for MRPC

    if args.mode == "train":
        optimizer = torch.optim.AdamW(model.parameters(), lr=config["learning_rate"])
        lr_scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * config["num_epochs"]
        )
        model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
            model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
        )

        for epoch in range(config["num_epochs"]):
            train_loss = training(accelerator, model, tokenizer, train_dataloader, optimizer, lr_scheduler, config, args)
            eval_metric = evaluation(accelerator, model, eval_dataloader, metric)
            accelerator.print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Eval Metric = {eval_metric}")
            accelerator.wait_for_everyone()
            unwrapped_model = accelerator.unwrap_model(model)
            accelerator.save(
                {
                    "model": unwrapped_model.state_dict(),
                    "optimizer": optimizer.state_dict(),
                    "lr_scheduler": lr_scheduler.state_dict(),
                    "epoch": epoch
                },
                os.path.join(args.output_dir, f"checkpoint_epoch_{epoch}.pt")
            )

    elif args.mode == "evaluate":
        if not args.checkpoint:
            raise ValueError("Checkpoint path is required for evaluation.")
        model.load_state_dict(torch.load(args.checkpoint)["model"])
        model = accelerator.prepare(model)
        eval_metric = evaluation(accelerator, model, eval_dataloader, metric)
        accelerator.print(f"Evaluation Metric: {eval_metric}")

    elif args.mode == "test":
        if not args.checkpoint:
            raise ValueError("Checkpoint path is required for testing.")
        test_dataloader = DataLoader(tokenized_datasets["test"], batch_size=config["batch_size"])
        model = AutoModelForSequenceClassification.from_pretrained(args.model_name, num_labels=2)
        model.load_state_dict(torch.load(args.checkpoint)["model"])
        model, test_dataloader = accelerator.prepare(model, test_dataloader)
        predictions = testing(model, test_dataloader, accelerator)
        print(f"Test Predictions: {predictions}")

if __name__ == "__main__":
    main()
```
